In [1]:
import simpy
from random import expovariate, seed

In [2]:
# global constants
N_PROCESSORS = 1
N_REQUESTS = 2000
MAX_QUEUE_LEN = 100
AVG_REQUEST_INTERVAL = 700
AVG_SERVING_TIME = 100
MAX_TIME = 30000
SEED = 42

payload_balance = {
    30000: {
        0.1: {'a': 700, 'b': 100},
        0.9: {'a': 191, 'b': 100},
    },
    20000: {
        0.1: {'a': 1100, 'b': 100},
        0.9: {'a': 191, 'b': 100},
    },
    10000: {
        0.1: {'a': 450, 'b': 100},
        0.9: {'a': 210, 'b': 100},
    }
}

test_times = [2000, 4000, 6000, 8000, 10000, 20000, 30000]

# monitoring variables

In [3]:
class ProcessMonitor:
    ''' Collects the data about running inside process. 
    
    This class does not pretend to be general by any means.
    '''
    def __init__(self, env, resource):
        self.env = env
        self.resource = resource
        
        self._proc_working_time = 0
        self._serving_time = []
        self._waiting_time = []
        self._num_rejected = 0
        
    def collect_data(self, n_requests, avg_request_interval, avg_serving_time, max_time):
        self.n_requests = n_requests
        self._avg_request_interval = avg_request_interval
        self._avg_serving_time = avg_serving_time
        
        self.env.process(self._generate_requests())
        self.env.run(until=max_time)
        
        return {
            'w': self._waiting_time, 
            'b': self._serving_time,
            'p': self._proc_working_time,
            'n_rejected': self._num_rejected,
        }
    
    def _generate_requests(self):
        for i in range(self.n_requests):
            self.env.process(self._serve())
            t = expovariate(1.0/self._avg_request_interval)
            yield self.env.timeout(t) # wait for the next request to appear      
        
    def _serve(self):
        num_rejected = 0
        arrive = self.env.now
        
        if len(self.resource.queue) < MAX_QUEUE_LEN:
            with self.resource.request() as req:
                yield req
                wait_time = self.env.now-arrive
                self._waiting_time.append(wait_time)

                service_time = expovariate(1.0/AVG_SERVING_TIME)
                self._serving_time.append(service_time)
                yield self.env.timeout(service_time) # wait to be served

            self._proc_working_time += env.now - arrive 
        else:
            num_rejected += 1

In [4]:
seed(SEED)

results = {}

for max_time, payload_params in payload_balance.items():
    results[max_time] = {}
    for payload, values in payload_params.items():
        env = simpy.Environment(0)
        k = simpy.Resource(env, capacity=N_PROCESSORS)
        m = ProcessMonitor(env, k)
        results[max_time][payload] = m.collect_data(N_REQUESTS, values['a'], values['b'], max_time)

In [5]:
time_ = 20000
print("Загрузка={}".format(results[time_][0.1]['p'] / time_))
print("Загрузка={}".format(results[time_][0.9]['p'] / time_))

Загрузка=0.11265890030034979
Загрузка=0.9694710480187868
